# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [2]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
#import ollama



# load_dotenv(override=True)
# openrouter_api_key = os.getenv('Open_Router_Key')
# print(openrouter_api_key)

In [37]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('Open_Router_Key')

if api_key and api_key.startswith('sk-or-v1') and len(api_key)>10:
   print("API key looks good so far")
else:
   print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
#MODEL = 'gpt-4o-mini'
#openai = OpenAI()

#MODEL = 'deepseek-r1:1.5b'
#openai = OpenAI(
#    base_url="http://localhost:11434/v1",
#    api_key="ollama"
#)
MODEL = 'google/gemini-2.0-flash-lite-preview-02-05:free'
MODEL = 'google/gemini-2.0-flash-thinking-exp:free'
MODEL ='google/gemini-2.0-pro-exp-02-05:free'
#MODEL ='qwen/qwen2.5-vl-72b-instruct:free'
#MODEL ='deepseek/deepseek-r1-distill-llama-70b:free'

openai = OpenAI(
    api_key=api_key,
    base_url="https://openrouter.ai/api/v1"
)



response = openai.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": "Write a short poem about today."}]
    )
print(response.choices[0].message.content)

API key looks good so far
The sun rose up, a gentle hue,
Painting skies in shades of blue.
A breeze went whispering through the trees,
Carrying scents on the morning ease.

The day unfolds, a quiet scroll,
With stories whispered, taking hold.
A simple moment, held so dear,
Today is here, and I am here.



In [5]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [8]:
mysite = Website("https://stoneskin.github.io/")
#print(mysite.get_contents())
print(mysite.links)

['http://mlccc.org', '/Scratch/', 'http://www.mlccc.org/', '/python/', 'http://www.mlccc.org/', 'https://stoneskin.github.io/learnJava/', 'http://www.mlccc.org/', '/SummerCamp_Scratch/', '/SummerCamp_Python_PyGame/', '/Java/', '/SummerCamp_Scratch/', '/SummerCamp_Python_Minecraft/', 'https://scratch.mit.edu/studios/3018337/projects/', './samples/index.html', 'https://1drv.ms/u/s!Avm-0LBz8-Jhg4M4HD4vOYb3PJFYUA?e=2ZGBLU', '/Tools/Git.01-LearnToUseGitForVersionControl.html', '/Tools/Git.02-UserSmartGitAsGitClient.html', '/Tools/Git.03-UseGitHubSite.html', '/python/4_Github/4.1_GithubAndVSCode.html', 'https://stoneskin.github.io/AirForce/', 'https://github.com/stoneskin/AirForce', 'https://stoneskin.github.io/Demo/CCRApp/login.html', 'https://github.com/MlcccCodingClass/ChineseCharactersRecognitionApp', 'https://stoneskin.github.io/GetStartPhp/', 'https://github.com/stoneskin/GetStartPhp', 'https://stoneskin.github.io/python-minecraft', 'https://stoneskin.github.io/mcpi-e/', 'https://stone

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
print(get_links_user_prompt(mysite))

Here is the list of links on the website of https://stoneskin.github.io/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
http://mlccc.org
/Scratch/
http://www.mlccc.org/
/python/
http://www.mlccc.org/
https://stoneskin.github.io/learnJava/
http://www.mlccc.org/
/SummerCamp_Scratch/
/SummerCamp_Python_PyGame/
/Java/
/SummerCamp_Scratch/
/SummerCamp_Python_Minecraft/
https://scratch.mit.edu/studios/3018337/projects/
./samples/index.html
https://1drv.ms/u/s!Avm-0LBz8-Jhg4M4HD4vOYb3PJFYUA?e=2ZGBLU
/Tools/Git.01-LearnToUseGitForVersionControl.html
/Tools/Git.02-UserSmartGitAsGitClient.html
/Tools/Git.03-UseGitHubSite.html
/python/4_Github/4.1_GithubAndVSCode.html
https://stoneskin.github.io/AirForce/
https://github.com/stoneskin/AirForce
https://stoneskin.github.io/Demo/CCRApp/login.html
https://github.com/MlcccC

In [10]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [35]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..
#target_url = "https://stoneskin.github.io/"
target_url = "https://huggingface.co"
huggingface = Website(target_url)
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/blog/inference-providers',
 '/spaces',
 '/models',
 '/deepseek-ai/DeepSeek-R1',
 '/Zyphra/Zonos-v0.1-hybrid',
 '/agentica-org/DeepScaleR-1.5B-Preview',
 '/hexgrad/Kokoro-82M',
 '/Zyphra/Zonos-v0.1-transformer',
 '/models',
 '/spaces/black-forest-labs/FLUX.1-dev',
 '/spaces/tencent/Hunyuan3D-2',
 '/spaces/deepseek-ai/Janus-Pro-7B',
 '/spaces/akhaliq/anychat',
 '/spaces/webml-community/kokoro-webgpu',
 '/spaces',
 '/datasets/open-r1/OpenR1-Math-220k',
 '/datasets/open-thoughts/OpenThoughts-114k',
 '/datasets/saiyan-world/Goku-MovieGenBench',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/Anthropic/EconomicIndex',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '

In [39]:
#get_links("https://huggingface.co")
get_links(target_url)

{'links': [{'type': 'company page',
   'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'linkedin page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [23]:
def get_all_details(url):
    print("get_all_details:", url)   
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [40]:
print(get_all_details(target_url))

get_all_details: https://huggingface.co
Found links: {'links': [{'type': 'company page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'brand assets', 'url': 'https://huggingface.co/brand'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
Welcome to Inference Providers on the Hub 🔥
smolagents - a smol library to build great agents
Use models from the HF Hub in LM Studio
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
deepseek-ai/DeepSeek-R1
Updated
6 days ago
•
3.71M
•
8.88k

In [16]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [17]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [21]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'}, {'type': 'models', 'url': 'https://huggingface.co/models'}, {'type': 'datasets', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces', 'url': 'https://huggingface.co/spaces'}, {'type': 'posts', 'url': 'https://huggingface.co/posts'}, {'type': 'docs', 'url': 'https://huggingface.co/docs'}, {'type': 'enterprise', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'careers/jobs', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'learn', 'url': 'https://huggingface.co/learn'}, {'type': 'status', 'url': 'https://status.huggingface.co/'}, {'type': 'github', 'url': 'https://github.com/huggingface'}, {'type': 'twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'discord', 'url': 'https://hug

'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nWelcome to Inference Providers on the Hub 🔥\nsmolagents - a smol library to build great agents\nUse models from the HF Hub in LM Studio\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ndeepseek-ai/DeepSeek-R1\nUpdated\n6 days ago\n•\n3.71M\n•\n8.88k\nZyphra/Zonos-v0.1-hybrid\nUpdated\n2 days ago\n•\n5.32k\n•\n763\nagentica-org/DeepScaleR-1.5B-Preview\nUpdated\n4 days ago\n•\n6.65k\n•\n319\nhexgrad/Kokoro-82M\nUpdated\n13 days ago\n•\n5

In [19]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [41]:
create_brochure("HuggingFace", "https://huggingface.co")

get_all_details: https://huggingface.co
Found links: {'links': [{'type': 'enterprise', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing', 'url': 'https://huggingface.co/pricing'}, {'type': 'brand', 'url': 'https://huggingface.co/brand'}, {'type': 'careers', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'about', 'url': 'https://huggingface.co/huggingface'}]}


# Hugging Face: The AI Community Building the Future

**Empowering Innovation in Machine Learning**

Hugging Face is the leading platform for the machine learning community, fostering collaboration on models, datasets, and applications. We're building the foundation of ML tooling, empowering individuals and organizations to create, discover, and deploy cutting-edge AI.

## What We Offer

*   **The Collaboration Platform:** A central hub to host and collaborate on unlimited public models, datasets, and applications. Explore a vast library of over 1 million models, 250,000 datasets and 400,000 applications across various modalities, including text, image, video, audio, and 3D.
*   **Open Source Stack:** Benefit from a rich ecosystem of powerful, open-source libraries:
    *   **Transformers:** State-of-the-art ML for PyTorch, TensorFlow, and JAX.
    *   **Diffusers:** Cutting-edge diffusion models for image and audio generation.
    *   **Datasets:** Access and share datasets tailored for various tasks.
    *   Safetensors, Hub Python Library, Tokenizers, PEFT, Transformers.js, timm, TRL, Text Generation Inference, Accelerate.

*   **Compute Solutions:** Easy to use solutions
	*   **Inference Endpoints:** Deploy models.
	*   **Spaces:** Update applications to a Spaces GPU.
*   **Enterprise Hub:**
    For organizations seeking advanced capabilities, the Enterprise Hub offers:
    *   Enterprise-grade security
    *   Access controls
    *   Dedicated support
    *   Single Sign-On
    *   Audit Logs
    *   Private Datasets Viewer

## Who We Serve

Over 50,000 organizations, including industry leaders, rely on Hugging Face:

*   AI2
*   Meta
*   Amazon Web Services
*   Google
*   Intel
*   Microsoft
*   Grammarly
*   Writer

## Our Culture
From the information provided it is difficult to fully ascertain the company culture, but certain assumptions can be made.
*    Open Source: The company is dedicatd to open source principles and collaboration.
*    Community Focused: Building tools for the Machine Learning community.
*    Innovation: Focused on state of the are and cutting edge technology.

## Careers

Hugging Face has a dedicated jobs page on their website, but no specific positions are listed here.

**Join the AI Revolution**

Whether you're an individual developer, a researcher, or part of a large enterprise, Hugging Face provides the tools and community to accelerate your machine-learning journey. Sign up, subscribe to an Enterprise Hub and start building the future of AI.


## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [42]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [43]:
stream_brochure("MLCCC", "https://mlccc.herokuapp.com/")

get_all_details: https://mlccc.herokuapp.com/
Found links: {'links': [{'type': 'about page', 'url': 'https://mlccc.herokuapp.com/about.html'}, {'type': 'programs page', 'url': 'https://mlccc.herokuapp.com/program.html'}, {'type': 'camps page', 'url': 'https://mlccc.herokuapp.com/camps.html'}, {'type': 'community page', 'url': 'https://mlccc.herokuapp.com/community.html'}, {'type': 'support page', 'url': 'https://mlccc.herokuapp.com/support.html'}, {'type': 'press page', 'url': 'https://mlccc.herokuapp.com/press.html'}, {'type': 'newsletter page', 'url': 'https://mlccc.herokuapp.com/newsletter.html'}, {'type': 'testimonials page', 'url': 'https://mlccc.herokuapp.com/testimonial.html'}]}


## MLCCC: Bridging Cultures, Building Futures

**Welcome to the Main Line Chinese Culture Center (MLCCC)!**

Established in 2003, MLCCC is a non-profit organization dedicated to fostering understanding between American and Chinese cultures in the Greater Philadelphia area. We offer a vibrant learning environment for students of all ages and backgrounds, providing compelling insights into Chinese language, arts, and traditions.

### What We Offer:

*   **Weekend School:** Our core program, offering comprehensive Chinese language instruction (Mandarin) from Pre-K to 12th grade, alongside enriching classes in traditional Chinese arts like dance, painting, and self-defense. Classes are held in person at GVHS, with some options available to connect via Zoom.
*   **Summer Camp:** An immersive experience for children, combining Chinese language learning with fun and engaging cultural activities. The camp runs for 9 weeks with flexible registration options.
*   **Community Engagement:** We are more than just a school – we are a conduit for cultural exchange.
Partnering with organizations like ABD international.

### Our Mission & Values:

*   **Mission:** To provide compelling insights into Chinese culture, supporting the educational needs and career aspirations of our community.
*   **Teaching Philosophy:** We emphasize traditional Asian teaching styles, fostering strong study skills and discipline.
*   **Community Focus:** We are a welcoming community built on parent involvement, volunteerism, and a shared passion for Chinese culture. Our PTO enhances the cultural experience and helps.

### Why Choose MLCCC?

*   **Proven Track Record:** From 30 students in 2003 to over 300 by 2008, our growth is a testament to the value of our programs.
*   **Experienced and passionate Instructors.**
*   **Diverse and Inclusive:** We welcome families from all backgrounds, with a current student body that is 87% Chinese-American.
*Interm chairman of the board is Qunbin Xiong, and the Principal is Julia Chou.
*   **Accreditation:** PVSA.
*   **Community Support:** We are supported by sponsors, parent volunteers, and a dedicated Board of Directors.

**Testimonial:**

"We love MLCCC! My kids not only improved their Chinese, but also enjoyed all kinds of cultural activities held at MLCCC. The teachers are great, very loving and encouraging, and very responsible..." - Z.C.

### Career Opportunities:

MLCCC occationally has part time and full time positions.
Check the website for more details.

### Contact Us:

*   **Address (Sundays only):** 225 N. Phoenixville Pike, Malvern, PA 19355
*   **Mailing Address:** P. O. Box 745, Bryn Mawr, PA 19010
*   **Phone:** +1-610-909-6094
*   **Email:** info@mlccc.org

**Join us at MLCCC and embark on a journey of cultural discovery!**


In [44]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

stream_brochure("HuggingFace", "https://huggingface.co")

get_all_details: https://huggingface.co
Found links: {'links': [{'type': 'company page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'brand assets', 'url': 'https://huggingface.co/brand'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


## Hugging Face: Where AI Gets a Big, Warm Hug! 🤗

**(And You Can, Too!)**

Are you tired of AI being all cold and robotic? Do you dream of a world where machine learning models, datasets, and applications all hang out and collaborate like one big, happy family? Then welcome to **Hugging Face**, the AI community that's building the future, one hug at a time!

**(Disclaimer: Actual hugging of AI models not recommended. May void warranty. And confuse the robots.)**

### What We Do (Besides Hugging, Obviously)

We're the go-to platform for the machine learning community. Think of us as the ultimate AI playground where over 1 million+ models, 250K+ datasets, and 400K+ applications are just waiting for you to explore them. Whether you're into text, images, video, audio, or even 3D (because who isn't?), we've got you covered.

**Trending Now!** (Like, *really* trending):

*   **DeepSeek-R1:** It's deep, it seeks, and it's got millions of downloads. What more could you want?
*   **Zonos-v0.1-hybrid:** So new. So Fresh.
*  **Kokoro-82M:** This must be the place.

### For the Lone Wolf (or the Pack Leader)

*   **Build Your Portfolio:** Show off your ML skills and become the envy of all your AI-enthusiast friends.
*   **Move Faster:** With our open-source stack!
*   **Unlimited Possibilities:** unlimited public models, datasets, and applications.

### Join the Hugging Face Family! (We're More Than Just a Name)

More than 50,000 organizations are already part of the Hugging Face community. We're talking big names like:

*   **Meta:** You know, the folks that may or not be tracking your every movement online.
*   **Google:** They know what you searched for last summer. And the summer before that.
*   **Microsoft:** Keeping your Clippy dreams alive, one AI model at a time.
*   **Amazon Web Services, Intel and Grammarly:** Also in the gang.

**(We're not saying we're influential, but... okay, we're totally saying it.)**

### Our Open Source Arsenal

We're not just about hugging; we're also about building! Check out our open-source tools. They include:

*   **Transformers:** "State of the Art" sounds impressive right?
*   **Diffusers**: "For generating images" sounds pretty cool too!
* And other fun things with exciting tech names such as; **Safetensors, Tokenizers, PEFT, Accelerate** and **Text Generation Inference**.

### Careers at Hugging Face: Where Your Quirks are Celebrated!

**(Check the Jobs page – link conveniently located elsewhere because we ran out of space in this brochure!)**
We are always looking for amazing people to come build the future of AI with us
If you like, you could be...
* Building the platform used by the leading Machine Learning teams in new and creative ways.
* Building and operating the infrastructure that enables running the largest models on the planet, and training models for our research team and customers.
* Collaborating with top AI researchers.

### In Conclusion...

Hugging Face is more than just a platform; it's a movement! A movement towards a more collaborative, open, and yes, *huggable* AI future. So, what are you waiting for? Come join the fun!

**(Warning: Side effects of joining Hugging Face may include increased knowledge of machine learning, an overwhelming urge to collaborate, and a newfound appreciation for the word "hugging.")**

**Sign up today! It's free, it's fun, and it's way better than talking to that chatbot that keeps trying to sell you insurance.**


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>